##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrating tf.summary usage to TF 2.x

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/migrate"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/migrate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/migrate.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub에서 소스 보기</a></td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/migrate.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

> Note: This doc is for people who are already familiar with TensorFlow 1.x TensorBoard and who want to migrate large TensorFlow code bases from TensorFlow 1.x to 2.x. If you're new to TensorBoard, see the [get started](get_started.ipynb) doc instead. If you are using `tf.keras` there may be no action you need to take to upgrade to TensorFlow 2.x. 


In [ ]:
import tensorflow as tf

TensorFlow 2.x includes significant changes to the `tf.summary` API used to write summary data for visualization in TensorBoard.

## 변경된 사항

`tf.summary` API를 두 개의 하위 API로 생각하면 편리합니다.

- 모델 코드에서 인라인으로 호출되는 `summary.scalar()`, `summary.histogram()`, `summary.image()`, `summary.audio()`, `summary.text()` 등 개별 요약을 기록하기 위한 ops 세트
- 이러한 개별 요약을 수집하고 특수 형식의 로그 파일에 기록하는(그런 다음 TensorBoard가 읽고 시각화를 생성함) 논리 작성

### TF 1.x에서

The two halves had to be manually wired together - by fetching the summary op outputs via `Session.run()` and calling `FileWriter.add_summary(output, step)`.  The `v1.summary.merge_all()` op made this easier by using a graph collection to aggregate all summary op outputs, but this approach still worked poorly for eager execution and control flow, making it especially ill-suited for TF 2.x.

### TF 2.X에서

두 부분이 긴밀하게 통합되었으며 이제 개별 `tf.summary` ops가 실행되는 즉시 데이터를 기록합니다. 모델 코드에서 API를 사용하는 것이 여전히 익숙해 보이겠지만 이제는 그래프 모드 호환성을 유지하면서 즉시 실행하기가 원활해졌습니다. API의 두 부분이 통합되면서 `summary.FileWriter`는 이제 TensorFlow 실행 컨텍스트의 일부가 되었고 `tf.summary` ops에서 직접 액세스하므로 작성기를 구성하는 것이 가장 큰 차이점으로 보일 것입니다.

Example usage with eager execution, the default in TF 2.x:

In [ ]:
writer = tf.summary.create_file_writer("/tmp/mylogs/eager")

with writer.as_default():
  for step in range(100):
    # other model code would go here
    tf.summary.scalar("my_metric", 0.5, step=step)
    writer.flush()

In [ ]:
ls /tmp/mylogs/eager

다음은 tf.function 그래프 실행을 사용한 예입니다.

In [ ]:
writer = tf.summary.create_file_writer("/tmp/mylogs/tf_function")

@tf.function
def my_func(step):
  with writer.as_default():
    # other model code would go here
    tf.summary.scalar("my_metric", 0.5, step=step)

for step in tf.range(100, dtype=tf.int64):
  my_func(step)
  writer.flush()

In [ ]:
ls /tmp/mylogs/tf_function

다음은 레거시 TF 1.x 그래프 실행의 사용 예입니다.


In [ ]:
g = tf.compat.v1.Graph()
with g.as_default():
  step = tf.Variable(0, dtype=tf.int64)
  step_update = step.assign_add(1)
  writer = tf.summary.create_file_writer("/tmp/mylogs/session")
  with writer.as_default():
    tf.summary.scalar("my_metric", 0.5, step=step)
  all_summary_ops = tf.compat.v1.summary.all_v2_summary_ops()
  writer_flush = writer.flush()


with tf.compat.v1.Session(graph=g) as sess:
  sess.run([writer.init(), step.initializer])

  for i in range(100):
    sess.run(all_summary_ops)
    sess.run(step_update)
    sess.run(writer_flush)  

In [ ]:
ls /tmp/mylogs/session

## 코드 변환하기

Converting existing `tf.summary` usage to the TF 2.x API cannot be reliably automated, so the [`tf_upgrade_v2` script](https://www.tensorflow.org/guide/upgrade) just rewrites it all to `tf.compat.v1.summary` and will not enable the TF 2.x behaviors automatically.

### Partial Migration

To make migration to TF 2.x easier for users of model code that still depends heavily on the TF 1.x summary API logging ops like `tf.compat.v1.summary.scalar()`, it is possible to migrate only the writer APIs first, allowing for individual TF 1.x summary ops inside your model code to be fully migrated at a later point.

To support this style of migration, <a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/summary"><code>tf.compat.v1.summary</code></a> will automatically forward to their TF 2.x equivalents under the following conditions:

 - The outermost context is eager mode
 - A default TF 2.x summary writer has been set
 - A non-empty value for step has been set for the writer (using <a href="https://www.tensorflow.org/api_docs/python/tf/summary/SummaryWriter#as_default"><code>tf.summary.SummaryWriter.as_default</code></a>, <a href="https://www.tensorflow.org/api_docs/python/tf/summary/experimental/set_step"><code>tf.summary.experimental.set_step</code></a>, or alternatively <a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/create_global_step"><code>tf.compat.v1.train.create_global_step</code></a>)

Note that when TF 2.x summary implementation is invoked, the return value will be an empty bytestring tensor, to avoid duplicate summary writing. Additionally, the input argument forwarding is best-effort and not all arguments will be preserved (for instance `family` argument will be supported whereas `collections` will be removed).

Example to invoke <a href="https://www.tensorflow.org/api_docs/python/tf/summary/scalar"><code>tf.summary.scalar</code></a> behaviors in <a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/summary/scalar"><code>tf.compat.v1.summary.scalar</code></a>:

In [ ]:
# Enable eager execution.
tf.compat.v1.enable_v2_behavior()

# A default TF 2.x summary writer is available.
writer = tf.summary.create_file_writer("/tmp/mylogs/enable_v2_in_v1")
# A step is set for the writer.
with writer.as_default(step=0):
  # Below invokes `tf.summary.scalar`, and the return value is an empty bytestring.
  tf.compat.v1.summary.scalar('float', tf.constant(1.0), family="family")

### Full Migration

To fully migrate to TF 2.x, you'll need to adapt your code as follows:

1. 요약 ops를 사용하려면 `.as_default()`를 통해 설정된 기본 작성기가 있어야 합니다.

    - 즉, ops를 즉시 실행하거나 ops를 그래프 구성에서 사용합니다.
    - 기본 작성기가 없으면 요약 ops가 자동 ops가 됩니다.
    - 기본 작성자는 (아직) `@tf.function` 실행 경계를 넘어 전파되지 않으며 함수가 추적될 때만 감지됩니다. 따라서 함수 본문 내에서 `writer.as_default()`를 호출하고 `@tf.function`이 사용되는 동안 작성기 객체가 계속해서 존재하도록 하는 것이 최선의 방법입니다.

2. `step` 인수를 통해 "스텝" 값을 각 op로 전달해야 합니다.

    - TensorBoard에서 데이터를 시계열로 렌더링하려면 스텝 값이 필요합니다.
    - TF 1.x의 전역 스텝이 제거되었기 때문에 명시적 전달이 필요하므로 각 op가 읽어야 하는 스텝 변수를 알아야 합니다.
    - 상용구를 줄이기 위해 기본 스텝 값을 등록하기 위한 실험적 지원이 `tf.summary.experimental.set_step()`으로 제공되지만 이는 예고 없이 변경될 수 있는 잠정적 기능입니다.

3. 개별 요약 ops의 함수 서명이 변경되었습니다.

    - 반환 값은 이제 부울입니다(요약이 실제로 작성되었는지 여부를 나타냄).
    - 두 번째 매개변수 이름(사용된 경우)이 `tensor`에서 `data`로 변경되었습니다.
    - `collections` 매개변수가 제거되었습니다. collections는 TF 1.x 전용입니다.
    - `family` 매개변수가 제거되었습니다. `tf.name_scope()`를 사용하세요.

4. [레거시 그래프 모드/세션 실행 사용자만 해당]

    - 먼저, `v1.Session.run(writer.init())`로 작성기를 초기화합니다.

    - Use `v1.summary.all_v2_summary_ops()` to get all TF 2.x summary ops for the current graph, e.g. to execute them via `Session.run()`

    - `v1.Session.run(writer.flush())`로 작성기를 플러시하고 `close()`에 대해서도 플러시합니다.

If your TF 1.x code was instead using `tf.contrib.summary` API, it's much more similar to the TF 2.x API, so `tf_upgrade_v2` script will automate most of the migration steps (and emit warnings or errors for any usage that cannot be fully migrated).  For the most part it just rewrites the API calls to `tf.compat.v2.summary`; if you only need compatibility with TF 2.x you can drop the `compat.v2` and just reference it as `tf.summary`.

## 추가 팁

위의 중요한 부분들 외에도 일부 보완적인 변경 사항도 있습니다.

- 조건부 기록(예: "100개 스텝마다 기록")의 모습이 새롭게 바뀌었습니다.

    - ops 및 관련 코드를 제어하려면 일반 if 문(즉시 실행 모드 및 [autograph를 통해 `@tf.function`](https://www.tensorflow.org/alpha/guide/autograph)에서 동작) 또는 `tf. cond`에서 이를 래핑합니다.
    - 요약만 제어하려면 새로운 `tf.summary.record_if()` 컨텍스트 관리자를 사용하고 여기에 선택한 부울 조건을 전달합니다.
    - 다음은 TF 1.x 패턴을 대체합니다.
        ```
        if condition:
          writer.add_summary()
        ```


- `tf.compat.v1.Graph`를 직접 작성하지 않고 대신 추적 함수를 사용합니다.

    - Graph execution in TF 2.x uses `@tf.function` instead of the explicit Graph
    - In TF 2.x, use the new tracing-style APIs `tf.summary.trace_on()` and `tf.summary.trace_export()` to record executed function graphs


- 더 이상 `tf.summary.FileWriterCache`를 이용해 logdir별로 전역 작성기 캐싱을 수행하지 않습니다.

    - 사용자는 작성기 객체의 자체 캐싱/공유를 구현하거나 별도의 작성기를 사용해야 합니다(후자에 대한 TensorBoard 지원이 [진행 중](https://github.com/tensorflow/tensorboard/issues/1063)).


- 이벤트 파일 이진 표현이 변경되었습니다.

    - TensorBoard 1.x는 이미 새로운 형식을 지원합니다. 이 차이는 이벤트 파일에서 요약 데이터를 수동으로 구문 분석하는 사용자에게만 영향을 미칩니다.
    - 요약 데이터는 이제 텐서 바이트로 저장됩니다. `tf.make_ndarray(event.summary.value[0].tensor)`를 사용하여 이를 numpy로 변환할 수 있습니다.